In [1]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(32, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [2]:

from sshtunnel import SSHTunnelForwarder
for index, row in df.iterrows():
    doc_id = row.doc_id
    
    printstate = (("doc_id = %s") % (str(doc_id)))
    
    print("Current test: %s" % (str(printstate)))
    
    with SSHTunnelForwarder(('144.214.121.15', 22),
                                ssh_username='ninadt',
                                ssh_password='Ninad123',
                                remote_bind_address=('localhost', 3306),
                                local_bind_address=('localhost', 3300)):
        import UpdateDB as db
        case = db.checkOldGender(doc_id = doc_id)

    if case == False:

        print("Running: %12s" % (str(printstate)))

        import StyloGender as Stylo
        (predY, testY) = Stylo.getTestResults(doc_id = doc_id, glove = '../../glove/',
                                              dimensions = 50, chunk_size = 1000, batch_size = 10)
        
        loc = testY
        
        test_acc = predY[loc]
        
        test_bin = 0
        
        if(predY.tolist().index(max(predY)) == testY):
            test_bin = 1
        
        with SSHTunnelForwarder(('144.214.121.15', 22),
                                ssh_username='ninadt',
                                ssh_password='Ninad123',
                                remote_bind_address=('localhost', 3306),
                                local_bind_address=('localhost', 3300)):
            import UpdateDB as db
            case = db.updateresultOldGender(doc_id = doc_id, test_acc = test_acc, test_bin = test_bin)

        del Stylo

        from keras import backend as K
        K.clear_session()

        import time
        time.sleep(10)

        from IPython.display import clear_output
        clear_output()

    else:
        print("Skipped: %12s" % (str(printstate)))


In [3]:
%tb

No traceback available to show.
